In [ ]:
# 📦 Step 1: Install & Import Libraries
import pandas as pd
import os
from google.colab import files

# 📁 Step 2: Upload File
uploaded = files.upload()  # Upload your monthly ride-level CSV here

# Get the uploaded filename
file_name = list(uploaded.keys())[0]

# ✏️ Step 3: Define Config
CPZ_LAT_THRESHOLD = 40.7681                   # South of 60th St
CPZ_LNG_RANGE = (-74.02, -73.93)              # Approx Manhattan longitudes
TIME_FREQ = "1H"                              # Change to '15min', '1D', etc.
YEAR_MONTH = "2024_01"                        # Update this per file

# 📊 Step 4: Load, Filter, Create Time Series
def load_and_filter_cpz(file_path):
    df = pd.read_csv(file_path, parse_dates=["started_at"])

    # Filter stations in Manhattan CPZ only
    df_cpz = df[
        (df["start_lat"] < CPZ_LAT_THRESHOLD) &
        (df["start_lng"].between(*CPZ_LNG_RANGE))
    ].copy()

    return df_cpz


def create_time_series(df, freq="1H"):
    df["time_bucket"] = df["started_at"].dt.floor(freq)
    ts_df = df.groupby(["start_station_id", "time_bucket"]).size().reset_index(name="booking_count")
    return ts_df

# Run steps
df_cpz = load_and_filter_cpz(file_name)
ts_df = create_time_series(df_cpz, TIME_FREQ)

# 💾 Step 5: Save and Download
output_file = f"cpz_station_timeseries_{YEAR_MONTH}.csv"
ts_df.to_csv(output_file, index=False)
files.download(output_file)


MessageError: RangeError: Maximum call stack size exceeded.